In [15]:
# !pip install -r ./requirements.txt

In [2]:
# !pip uninstall virny -y

In [4]:
# Install using an HTTP link
# !pip install git+https://github.com/DataResponsibly/Virny.git@feature/prepare_for_uncertainty_experiments

# Install using an SSH link
# !pip install git+ssh://git@github.com/DataResponsibly/Virny.git@feature/prepare_for_uncertainty_experiments

In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [2]:
import os
import warnings
warnings.filterwarnings('ignore')
os.environ["PYTHONWARNINGS"] = "ignore"

In [3]:
cur_folder_name = os.getcwd().split('/')[-1]
if cur_folder_name != "fairness-variance":
    os.chdir("../../../..")

print('Current location: ', os.getcwd())

Current location:  /home/dh3553/projects/fairness-variance


## Import dependencies

In [4]:
import copy

from virny.utils.custom_initializers import create_config_obj
from virny.datasets import ACSPublicCoverageDataset

from configs.constants import TEST_SET_FRACTION, EXPERIMENT_SEEDS
from configs.models_config_for_tuning import get_folktables_employment_models_params_for_tuning2

from source.experiment_interface import run_exp_iter_with_LFR

pip install 'aif360[LawSchoolGPA]'
pip install 'aif360[AdversarialDebiasing]'
pip install 'aif360[AdversarialDebiasing]'


## Define Input Variables

In [5]:
ROOT_DIR = os.getcwd()
EXPERIMENT_NAME = 'LFR_acs_pubcov'
DB_COLLECTION_NAME = 'one_repair_lvl_many_models'
FAIR_INTERVENTION_PARAMS_LST = [{'k': 10, 'Ax': 0.1, 'Ay': 1.0, 'Az': 2.0}]
SAVE_RESULTS_DIR_PATH = os.path.join(ROOT_DIR, 'results', 'diff_fairness_interventions_exp', 'LFR', EXPERIMENT_NAME)

config_yaml_path = os.path.join(ROOT_DIR, 'notebooks', 'diff_fairness_interventions_exp',
                                'LFR', EXPERIMENT_NAME, 'folk_CA_2018_config.yaml')
metrics_computation_config = create_config_obj(config_yaml_path=config_yaml_path)

## Define a db writer and custom fields to insert into your database

In [6]:
import os
from dotenv import load_dotenv

load_dotenv('./configs/secrets.env')
os.getenv("DB_NAME")

'fairness_variance'

In [7]:
from source.utils.db_functions import connect_to_mongodb

client, collection_obj, db_writer_func = connect_to_mongodb(DB_COLLECTION_NAME)

In [8]:
import uuid

custom_table_fields_dct = {
#     'session_uuid': str(uuid.uuid4()),
    'session_uuid': '3021b0c7-483d-4682-9dc6-bb4a2cf90020',
}
print('Current session uuid: ', custom_table_fields_dct['session_uuid'])

Current session uuid:  3021b0c7-483d-4682-9dc6-bb4a2cf90020


## Initialize custom objects

In [9]:
data_loader = ACSPublicCoverageDataset(state=['CA'], year=2018, with_nulls=False,
                                       subsample_size=15_000, subsample_seed=42)
data_loader.X_data.head()

,SCHL,MAR,SEX,DIS,ESP,CIT,MIG,MIL,ANC,NATIVITY,DEAR,DEYE,DREM,ESR,ST,FER,RAC1P,AGEP,PINCP
0,19,5,1,2,0,1,3,4,1,1,2,2,2,6,6,0,1,21,3150.0
1,16,5,1,2,0,3,3,4,4,1,2,2,2,1,6,0,9,18,1600.0
2,13,5,2,2,1,1,1,0,2,1,2,2,2,6,6,2,1,16,0.0
3,20,1,2,2,0,4,1,4,1,2,2,2,2,6,6,2,8,43,0.0
4,16,1,2,2,0,4,1,4,1,2,2,2,2,6,6,0,6,54,0.0


In [10]:
data_loader.X_data.shape

(15000, 19)

## Run experiment iterations

### Experiment iteration 1

In [11]:
# tuned_params_filenames = ['tuning_results_Folktables_NY_2018_Employment_alpha_0.8_20230706__115508.csv']
# tuned_params_df_paths = [os.path.join(ROOT_DIR, 'results', EXPERIMENT_NAME, tuned_params_filename)
#                          for tuned_params_filename in tuned_params_filenames]

In [12]:
# Configs for an experiment iteration
exp_iter_num = 1
experiment_seed = EXPERIMENT_SEEDS[exp_iter_num - 1]
custom_table_fields_dct['experiment_iteration'] = f'Exp_iter_{exp_iter_num}'

exp_iter_data_loader = copy.deepcopy(data_loader)  # Add deepcopy to avoid data leakage
models_params_for_tuning = get_folktables_employment_models_params_for_tuning2(experiment_seed)

In [ ]:
run_exp_iter_with_LFR(data_loader=exp_iter_data_loader,
                      experiment_seed=experiment_seed,
                      test_set_fraction=TEST_SET_FRACTION,
                      db_writer_func=db_writer_func,
                      fair_intervention_params_lst=FAIR_INTERVENTION_PARAMS_LST,
                      models_params_for_tuning=models_params_for_tuning,
                      metrics_computation_config=metrics_computation_config,
                      custom_table_fields_dct=custom_table_fields_dct,
                      with_tuning=True,
                      # with_tuning=False,
                      # tuned_params_df_paths=tuned_params_df_paths,
                      save_results_dir_path=SAVE_RESULTS_DIR_PATH,
                      dataset_name='ACSPublicCoverageDataset',
                      verbose=True)

2024-01-02 17:51:41 experiment_interface.py INFO    : Start an experiment iteration for the following custom params:
INFO:root:Start an experiment iteration for the following custom params:


{'dataset_split_seed': 100,
 'experiment_iteration': 'Exp_iter_1',
 'fair_intervention_params_lst': "[{'k': 10, 'Ax': 0.1, 'Ay': 1.0, 'Az': 2.0}]",
 'model_init_seed': 100,
 'session_uuid': '3021b0c7-483d-4682-9dc6-bb4a2cf90020'}




Multiple alphas:   0%|          | 0/1 [00:00<?, ?it/s]

intervention_options:  {'k': 10, 'Ax': 0.1, 'Ay': 1.0, 'Az': 2.0}
step: 0, loss: 0.7738664492995504, L_x: 0.5312486415152264,  L_y: 0.7143212051500916,  L_z: 0.003210189998968106
step: 250, loss: 0.7738664455075007, L_x: 0.5312486589500682,  L_y: 0.7143211997095945,  L_z: 0.0032101899514496712
step: 500, loss: 0.7738664966445234, L_x: 0.5312486584956309,  L_y: 0.7143212508741544,  L_z: 0.0032101899604029703
step: 750, loss: 0.7738664400496074, L_x: 0.5312486541795671,  L_y: 0.7143211956873605,  L_z: 0.0032101894721450796
step: 1000, loss: 0.7392462514757174, L_x: 0.5309187861710835,  L_y: 0.6799026396761718,  L_z: 0.0031258665912186332
step: 1250, loss: 0.7392462476872592, L_x: 0.5309187672680289,  L_y: 0.6799026336300734,  L_z: 0.003125868665191446
step: 1500, loss: 0.7392462512138762, L_x: 0.5309187740814612,  L_y: 0.6799026408247633,  L_z: 0.0031258664904833906
step: 1750, loss: 0.7225249900779839, L_x: 0.5294238571872247,  L_y: 0.6638217072146866,  L_z: 0.002880448572287438
step: 2

2024-01-02 17:52:48 experiment_interface.py INFO    : The dataset is preprocessed
INFO:root:The dataset is preprocessed


cur_base_flow_dataset.X_train_val.columns:  Index(['cat__SCHL_1', 'cat__SCHL_10', 'cat__SCHL_11', 'cat__SCHL_12',
       'cat__SCHL_13', 'cat__SCHL_14', 'cat__SCHL_15', 'cat__SCHL_16',
       'cat__SCHL_17', 'cat__SCHL_18', 'cat__SCHL_19', 'cat__SCHL_2',
       'cat__SCHL_20', 'cat__SCHL_21', 'cat__SCHL_22', 'cat__SCHL_23',
       'cat__SCHL_24', 'cat__SCHL_3', 'cat__SCHL_4', 'cat__SCHL_5',
       'cat__SCHL_6', 'cat__SCHL_7', 'cat__SCHL_8', 'cat__SCHL_9',
       'cat__MAR_1', 'cat__MAR_2', 'cat__MAR_3', 'cat__MAR_4', 'cat__MAR_5',
       'cat__DIS_1', 'cat__DIS_2', 'cat__ESP_0', 'cat__ESP_1', 'cat__ESP_2',
       'cat__ESP_3', 'cat__ESP_4', 'cat__ESP_5', 'cat__ESP_6', 'cat__ESP_7',
       'cat__ESP_8', 'cat__CIT_1', 'cat__CIT_2', 'cat__CIT_3', 'cat__CIT_4',
       'cat__CIT_5', 'cat__MIG_1', 'cat__MIG_2', 'cat__MIG_3', 'cat__MIL_0',
       'cat__MIL_1', 'cat__MIL_2', 'cat__MIL_3', 'cat__MIL_4', 'cat__ANC_1',
       'cat__ANC_2', 'cat__ANC_3', 'cat__ANC_4', 'cat__NATIVITY_1',
       'c

2024-01-02 17:55:35 experiment_interface.py INFO    : Models are tuned and saved to a file
INFO:root:Models are tuned and saved to a file


Analyze multiple models:   0%|          | 0/1 [00:00<?, ?it/s]

Classifiers testing by bootstrap:   0%|          | 0/200 [00:00<?, ?it/s]

### Experiment iteration 2

In [21]:
# Configs for an experiment iteration
exp_iter_num = 2
experiment_seed = EXPERIMENT_SEEDS[exp_iter_num - 1]
tuned_params_filenames = [
    'tuning_results_Folktables_CA_2018_Public_Coverage_20240102__225535.csv'
]
tuned_params_df_paths = [os.path.join(ROOT_DIR, 'results', 'diff_fairness_interventions_exp',
                                      'LFR', EXPERIMENT_NAME, tuned_params_filename)
                         for tuned_params_filename in tuned_params_filenames]
custom_table_fields_dct['experiment_iteration'] = f'Exp_iter_{exp_iter_num}'

exp_iter_data_loader = copy.deepcopy(data_loader)  # Add deepcopy to avoid data leakage
models_params_for_tuning = get_folktables_employment_models_params_for_tuning2(experiment_seed)

In [22]:
run_exp_iter_with_LFR(data_loader=exp_iter_data_loader,
                      experiment_seed=experiment_seed,
                      test_set_fraction=TEST_SET_FRACTION,
                      db_writer_func=db_writer_func,
                      fair_intervention_params_lst=FAIR_INTERVENTION_PARAMS_LST,
                      models_params_for_tuning=models_params_for_tuning,
                      metrics_computation_config=metrics_computation_config,
                      custom_table_fields_dct=custom_table_fields_dct,
                      # with_tuning=True,
                      with_tuning=False,
                      tuned_params_df_paths=tuned_params_df_paths,
                      save_results_dir_path=SAVE_RESULTS_DIR_PATH,
                      dataset_name='ACSPublicCoverageDataset',
                      verbose=True)

2024-01-02 14:09:14 experiment_interface.py INFO    : Start an experiment iteration for the following custom params:
INFO:root:Start an experiment iteration for the following custom params:


{'dataset_split_seed': 200,
 'experiment_iteration': 'Exp_iter_2',
 'fair_intervention_params_lst': "[{'k': 10, 'Ax': 0.1, 'Ay': 1.0, 'Az': 2.0}]",
 'intervention_param': "{'k': 10, 'Ax': 0.1, 'Ay': 1.0, 'Az': 2.0}",
 'model_init_seed': 200,
 'session_uuid': '638da338-6a90-44a7-a835-17f062b941fb'}


Multiple alphas:   0%|          | 0/1 [00:00<?, ?it/s]

intervention_options:  {'k': 10, 'Ax': 0.1, 'Ay': 1.0, 'Az': 2.0}
step: 0, loss: 0.7989742365933232, L_x: 1.0333738605431104,  L_y: 0.6789815789838769,  L_z: 0.008327635777567639
step: 250, loss: 0.7989741666340583, L_x: 1.0333738969712114,  L_y: 0.6789815068392818,  L_z: 0.008327635048827662
step: 500, loss: 0.7989742485844722, L_x: 1.0333738780784012,  L_y: 0.6789815929472827,  L_z: 0.00832763391467471
step: 750, loss: 0.6391934258825521, L_x: 1.0332909502036012,  L_y: 0.5194029750388243,  L_z: 0.00823067791168386
step: 1000, loss: 0.6391934637366692, L_x: 1.033290960107876,  L_y: 0.5194030101634384,  L_z: 0.008230678781221638
step: 1250, loss: 0.5228079191602457, L_x: 1.0314852347504928,  L_y: 0.403770397229017,  L_z: 0.007944499228089695
step: 1500, loss: 0.5228079383442957, L_x: 1.0314852594486816,  L_y: 0.4037704005509528,  L_z: 0.007944505924237373
step: 1750, loss: 0.5192984513523966, L_x: 1.0302606206214335,  L_y: 0.4009645894024503,  L_z: 0.007653899943901492
step: 2000, loss

2024-01-02 14:09:17 experiment_interface.py INFO    : The dataset is preprocessed
INFO:root:The dataset is preprocessed
2024-01-02 14:09:17 experiment_interface.py INFO    : Models config is loaded from the input file
INFO:root:Models config is loaded from the input file


...

[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGB

[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGB

[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGB

Classifiers testing by bootstrap:   0%|          | 0/200 [00:00<?, ?it/s]

Classifiers testing by bootstrap:   0%|          | 0/200 [00:00<?, ?it/s]

### Experiment iteration 3

In [23]:
# Configs for an experiment iteration
exp_iter_num = 3
experiment_seed = EXPERIMENT_SEEDS[exp_iter_num - 1]
tuned_params_filenames = [
    'tuning_results_Folktables_CA_2018_Public_Coverage_20240102__225535.csv'
]
tuned_params_df_paths = [os.path.join(ROOT_DIR, 'results', 'diff_fairness_interventions_exp',
                                      'LFR', EXPERIMENT_NAME, tuned_params_filename)
                         for tuned_params_filename in tuned_params_filenames]
custom_table_fields_dct['experiment_iteration'] = f'Exp_iter_{exp_iter_num}'

exp_iter_data_loader = copy.deepcopy(data_loader)  # Add deepcopy to avoid data leakage
models_params_for_tuning = get_folktables_employment_models_params_for_tuning2(experiment_seed)

In [24]:
run_exp_iter_with_LFR(data_loader=exp_iter_data_loader,
                      experiment_seed=experiment_seed,
                      test_set_fraction=TEST_SET_FRACTION,
                      db_writer_func=db_writer_func,
                      fair_intervention_params_lst=FAIR_INTERVENTION_PARAMS_LST,
                      models_params_for_tuning=models_params_for_tuning,
                      metrics_computation_config=metrics_computation_config,
                      custom_table_fields_dct=custom_table_fields_dct,
                      # with_tuning=True,
                      with_tuning=False,
                      tuned_params_df_paths=tuned_params_df_paths,
                      save_results_dir_path=SAVE_RESULTS_DIR_PATH,
                      dataset_name='ACSPublicCoverageDataset',
                      verbose=True)

2024-01-02 14:11:34 experiment_interface.py INFO    : Start an experiment iteration for the following custom params:
INFO:root:Start an experiment iteration for the following custom params:


{'dataset_split_seed': 300,
 'experiment_iteration': 'Exp_iter_3',
 'fair_intervention_params_lst': "[{'k': 10, 'Ax': 0.1, 'Ay': 1.0, 'Az': 2.0}]",
 'intervention_param': "{'k': 10, 'Ax': 0.1, 'Ay': 1.0, 'Az': 2.0}",
 'model_init_seed': 300,
 'session_uuid': '638da338-6a90-44a7-a835-17f062b941fb'}


Multiple alphas:   0%|          | 0/1 [00:00<?, ?it/s]

intervention_options:  {'k': 10, 'Ax': 0.1, 'Ay': 1.0, 'Az': 2.0}
step: 0, loss: 0.7979782676913761, L_x: 1.0323907659454599,  L_y: 0.6781985204253934,  L_z: 0.00827033533571835
step: 250, loss: 0.797978199748413, L_x: 1.0323908019191732,  L_y: 0.6781984500730722,  L_z: 0.008270334741711762
step: 500, loss: 0.7979782787217018, L_x: 1.0323907832154928,  L_y: 0.6781985335275081,  L_z: 0.00827033343632219
step: 750, loss: 0.6475774505477387, L_x: 1.0322746489837868,  L_y: 0.5280085073128342,  L_z: 0.008170739168262924
step: 1000, loss: 0.6475774873926531, L_x: 1.0322746589152256,  L_y: 0.5280085418657529,  L_z: 0.008170739817688817
step: 1250, loss: 0.5396801755981412, L_x: 1.0305529553397612,  L_y: 0.42116019193809506,  L_z: 0.007732344063034973
step: 1500, loss: 0.5396801895434374, L_x: 1.0305529807322427,  L_y: 0.42116019036751473,  L_z: 0.007732350551349179
step: 1750, loss: 0.5361632824016606, L_x: 1.0293364091548245,  L_y: 0.41834810283738577,  L_z: 0.007440769324396114
step: 2000, 

2024-01-02 14:11:37 experiment_interface.py INFO    : The dataset is preprocessed
INFO:root:The dataset is preprocessed
2024-01-02 14:11:37 experiment_interface.py INFO    : Models config is loaded from the input file
INFO:root:Models config is loaded from the input file


...

[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGB

[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGB

[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGB

Classifiers testing by bootstrap:   0%|          | 0/200 [00:00<?, ?it/s]

Classifiers testing by bootstrap:   0%|          | 0/200 [00:00<?, ?it/s]

### Experiment iteration 4

In [13]:
# Configs for an experiment iteration
exp_iter_num = 4
experiment_seed = EXPERIMENT_SEEDS[exp_iter_num - 1]
tuned_params_filenames = [
    'tuning_results_Folktables_CA_2018_Public_Coverage_20240102__225535.csv'
]
tuned_params_df_paths = [os.path.join(ROOT_DIR, 'results', 'diff_fairness_interventions_exp',
                                      'LFR', EXPERIMENT_NAME, tuned_params_filename)
                         for tuned_params_filename in tuned_params_filenames]
custom_table_fields_dct['experiment_iteration'] = f'Exp_iter_{exp_iter_num}'

exp_iter_data_loader = copy.deepcopy(data_loader)  # Add deepcopy to avoid data leakage
models_params_for_tuning = get_folktables_employment_models_params_for_tuning2(experiment_seed)

In [ ]:
run_exp_iter_with_LFR(data_loader=exp_iter_data_loader,
                      experiment_seed=experiment_seed,
                      test_set_fraction=TEST_SET_FRACTION,
                      db_writer_func=db_writer_func,
                      fair_intervention_params_lst=FAIR_INTERVENTION_PARAMS_LST,
                      models_params_for_tuning=models_params_for_tuning,
                      metrics_computation_config=metrics_computation_config,
                      custom_table_fields_dct=custom_table_fields_dct,
                      # with_tuning=True,
                      with_tuning=False,
                      tuned_params_df_paths=tuned_params_df_paths,
                      save_results_dir_path=SAVE_RESULTS_DIR_PATH,
                      dataset_name='ACSPublicCoverageDataset',
                      verbose=True)

2024-01-02 18:06:37 experiment_interface.py INFO    : Start an experiment iteration for the following custom params:
INFO:root:Start an experiment iteration for the following custom params:


{'dataset_split_seed': 400,
 'experiment_iteration': 'Exp_iter_4',
 'fair_intervention_params_lst': "[{'k': 10, 'Ax': 0.1, 'Ay': 1.0, 'Az': 2.0}]",
 'model_init_seed': 400,
 'session_uuid': '3021b0c7-483d-4682-9dc6-bb4a2cf90020'}




Multiple alphas:   0%|          | 0/1 [00:00<?, ?it/s]

intervention_options:  {'k': 10, 'Ax': 0.1, 'Ay': 1.0, 'Az': 2.0}
step: 0, loss: 0.774309360558032, L_x: 0.5311950653962298,  L_y: 0.7145198353026864,  L_z: 0.003335009357861357
step: 250, loss: 0.7743093564558489, L_x: 0.5311950826031373,  L_y: 0.7145198297210035,  L_z: 0.0033350092372658375
step: 500, loss: 0.7743094089424412, L_x: 0.5311950823636551,  L_y: 0.714519881997667,  L_z: 0.0033350093542043255
step: 750, loss: 0.7743093511719942, L_x: 0.5311950780743788,  L_y: 0.7145198256792807,  L_z: 0.0033350088426377838
step: 1000, loss: 0.7384002156495854, L_x: 0.5308646467678899,  L_y: 0.6788221278416133,  L_z: 0.0032458115655915193
step: 1250, loss: 0.738400213050336, L_x: 0.5308646278331008,  L_y: 0.6788221217914947,  L_z: 0.003245814237765589
step: 1500, loss: 0.7384002155735931, L_x: 0.5308646346646294,  L_y: 0.6788221288783911,  L_z: 0.00324581161436954
step: 1750, loss: 0.7209773313421624, L_x: 0.5293431795133442,  L_y: 0.6621518097252077,  L_z: 0.0029456018328101142
step: 2000,

2024-01-02 18:07:37 experiment_interface.py INFO    : The dataset is preprocessed
INFO:root:The dataset is preprocessed
2024-01-02 18:07:37 experiment_interface.py INFO    : Models config is loaded from the input file
INFO:root:Models config is loaded from the input file


cur_base_flow_dataset.X_train_val.columns:  Index(['cat__SCHL_1', 'cat__SCHL_10', 'cat__SCHL_11', 'cat__SCHL_12',
       'cat__SCHL_13', 'cat__SCHL_14', 'cat__SCHL_15', 'cat__SCHL_16',
       'cat__SCHL_17', 'cat__SCHL_18', 'cat__SCHL_19', 'cat__SCHL_2',
       'cat__SCHL_20', 'cat__SCHL_21', 'cat__SCHL_22', 'cat__SCHL_23',
       'cat__SCHL_24', 'cat__SCHL_3', 'cat__SCHL_4', 'cat__SCHL_5',
       'cat__SCHL_6', 'cat__SCHL_7', 'cat__SCHL_8', 'cat__SCHL_9',
       'cat__MAR_1', 'cat__MAR_2', 'cat__MAR_3', 'cat__MAR_4', 'cat__MAR_5',
       'cat__DIS_1', 'cat__DIS_2', 'cat__ESP_0', 'cat__ESP_1', 'cat__ESP_2',
       'cat__ESP_3', 'cat__ESP_4', 'cat__ESP_5', 'cat__ESP_6', 'cat__ESP_7',
       'cat__ESP_8', 'cat__CIT_1', 'cat__CIT_2', 'cat__CIT_3', 'cat__CIT_4',
       'cat__CIT_5', 'cat__MIG_1', 'cat__MIG_2', 'cat__MIG_3', 'cat__MIL_0',
       'cat__MIL_1', 'cat__MIL_2', 'cat__MIL_3', 'cat__MIL_4', 'cat__ANC_1',
       'cat__ANC_2', 'cat__ANC_3', 'cat__ANC_4', 'cat__NATIVITY_1',
       'c

Analyze multiple models:   0%|          | 0/1 [00:00<?, ?it/s]

Classifiers testing by bootstrap:   0%|          | 0/200 [00:00<?, ?it/s]

### Experiment iteration 5

In [27]:
# Configs for an experiment iteration
exp_iter_num = 5
experiment_seed = EXPERIMENT_SEEDS[exp_iter_num - 1]
tuned_params_filenames = [
    'tuning_results_Folktables_CA_2018_Public_Coverage_20240102__225535.csv'
]
tuned_params_df_paths = [os.path.join(ROOT_DIR, 'results', 'diff_fairness_interventions_exp',
                                      'LFR', EXPERIMENT_NAME, tuned_params_filename)
                         for tuned_params_filename in tuned_params_filenames]
custom_table_fields_dct['experiment_iteration'] = f'Exp_iter_{exp_iter_num}'

exp_iter_data_loader = copy.deepcopy(data_loader)  # Add deepcopy to avoid data leakage
models_params_for_tuning = get_folktables_employment_models_params_for_tuning2(experiment_seed)

In [28]:
run_exp_iter_with_LFR(data_loader=exp_iter_data_loader,
                      experiment_seed=experiment_seed,
                      test_set_fraction=TEST_SET_FRACTION,
                      db_writer_func=db_writer_func,
                      fair_intervention_params_lst=FAIR_INTERVENTION_PARAMS_LST,
                      models_params_for_tuning=models_params_for_tuning,
                      metrics_computation_config=metrics_computation_config,
                      custom_table_fields_dct=custom_table_fields_dct,
                      # with_tuning=True,
                      with_tuning=False,
                      tuned_params_df_paths=tuned_params_df_paths,
                      save_results_dir_path=SAVE_RESULTS_DIR_PATH,
                      dataset_name='ACSPublicCoverageDataset',
                      verbose=True)

2024-01-02 14:16:10 experiment_interface.py INFO    : Start an experiment iteration for the following custom params:
INFO:root:Start an experiment iteration for the following custom params:


{'dataset_split_seed': 500,
 'experiment_iteration': 'Exp_iter_5',
 'fair_intervention_params_lst': "[{'k': 10, 'Ax': 0.1, 'Ay': 1.0, 'Az': 2.0}]",
 'intervention_param': "{'k': 10, 'Ax': 0.1, 'Ay': 1.0, 'Az': 2.0}",
 'model_init_seed': 500,
 'session_uuid': '638da338-6a90-44a7-a835-17f062b941fb'}


Multiple alphas:   0%|          | 0/1 [00:00<?, ?it/s]

intervention_options:  {'k': 10, 'Ax': 0.1, 'Ay': 1.0, 'Az': 2.0}
step: 0, loss: 0.8000964558093502, L_x: 1.0320650821383393,  L_y: 0.6802994870863546,  L_z: 0.008295230254580854
step: 250, loss: 0.8000963906903515, L_x: 1.032065117326718,  L_y: 0.6802994200786068,  L_z: 0.008295229439536474
step: 500, loss: 0.8000964681341676, L_x: 1.0320650992688587,  L_y: 0.6802995004138729,  L_z: 0.008295228896704438
step: 750, loss: 0.6536690292061383, L_x: 1.031969089471765,  L_y: 0.53409937860604,  L_z: 0.008186370826460966
step: 1000, loss: 0.6536690630394447, L_x: 1.0319690979327079,  L_y: 0.5340994111791351,  L_z: 0.008186371033519452
step: 1250, loss: 0.5498835673882443, L_x: 1.030299316549742,  L_y: 0.43144221627644824,  L_z: 0.007705709728410874
step: 1500, loss: 0.5498835849308982, L_x: 1.0302993430188478,  L_y: 0.43144221903501456,  L_z: 0.007705715796999389
step: 1750, loss: 0.546021331900659, L_x: 1.0291095664929255,  L_y: 0.42834864138975803,  L_z: 0.0073808669308041905
step: 2000, lo

2024-01-02 14:16:14 experiment_interface.py INFO    : The dataset is preprocessed
INFO:root:The dataset is preprocessed
2024-01-02 14:16:14 experiment_interface.py INFO    : Models config is loaded from the input file
INFO:root:Models config is loaded from the input file


...

[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGB

[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGB

[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGB

Classifiers testing by bootstrap:   0%|          | 0/200 [00:00<?, ?it/s]

Classifiers testing by bootstrap:   0%|          | 0/200 [00:00<?, ?it/s]

### Experiment iteration 6

In [29]:
# Configs for an experiment iteration
exp_iter_num = 6
experiment_seed = EXPERIMENT_SEEDS[exp_iter_num - 1]
tuned_params_filenames = [
    'tuning_results_Folktables_CA_2018_Public_Coverage_20240102__225535.csv'
]
tuned_params_df_paths = [os.path.join(ROOT_DIR, 'results', 'diff_fairness_interventions_exp',
                                      'LFR', EXPERIMENT_NAME, tuned_params_filename)
                         for tuned_params_filename in tuned_params_filenames]
custom_table_fields_dct['experiment_iteration'] = f'Exp_iter_{exp_iter_num}'

exp_iter_data_loader = copy.deepcopy(data_loader)  # Add deepcopy to avoid data leakage
models_params_for_tuning = get_folktables_employment_models_params_for_tuning2(experiment_seed)

In [30]:
run_exp_iter_with_LFR(data_loader=exp_iter_data_loader,
                      experiment_seed=experiment_seed,
                      test_set_fraction=TEST_SET_FRACTION,
                      db_writer_func=db_writer_func,
                      fair_intervention_params_lst=FAIR_INTERVENTION_PARAMS_LST,
                      models_params_for_tuning=models_params_for_tuning,
                      metrics_computation_config=metrics_computation_config,
                      custom_table_fields_dct=custom_table_fields_dct,
                      # with_tuning=True,
                      with_tuning=False,
                      tuned_params_df_paths=tuned_params_df_paths,
                      save_results_dir_path=SAVE_RESULTS_DIR_PATH,
                      dataset_name='ACSPublicCoverageDataset',
                      verbose=True)

2024-01-02 14:18:23 experiment_interface.py INFO    : Start an experiment iteration for the following custom params:
INFO:root:Start an experiment iteration for the following custom params:


{'dataset_split_seed': 600,
 'experiment_iteration': 'Exp_iter_6',
 'fair_intervention_params_lst': "[{'k': 10, 'Ax': 0.1, 'Ay': 1.0, 'Az': 2.0}]",
 'intervention_param': "{'k': 10, 'Ax': 0.1, 'Ay': 1.0, 'Az': 2.0}",
 'model_init_seed': 600,
 'session_uuid': '638da338-6a90-44a7-a835-17f062b941fb'}


Multiple alphas:   0%|          | 0/1 [00:00<?, ?it/s]

intervention_options:  {'k': 10, 'Ax': 0.1, 'Ay': 1.0, 'Az': 2.0}
step: 0, loss: 0.8012308209526325, L_x: 1.032486691591748,  L_y: 0.6816260094522957,  L_z: 0.00817807117058102
step: 250, loss: 0.8012307592510307, L_x: 1.03248672607275,  L_y: 0.6816259452091471,  L_z: 0.00817807071730432
step: 500, loss: 0.8012308321514614, L_x: 1.032486709583512,  L_y: 0.6816260226759697,  L_z: 0.00817806925857021
step: 750, loss: 0.6601098419829545, L_x: 1.0323946558803412,  L_y: 0.5407171178798217,  L_z: 0.00807662925754936
step: 1000, loss: 0.66010987855932, L_x: 1.0323946661335703,  L_y: 0.5407171500610479,  L_z: 0.00807663094245753
step: 1250, loss: 0.5619558254545324, L_x: 1.0308029125968043,  L_y: 0.443589170867648,  L_z: 0.007643181663601961
step: 1500, loss: 0.5619558401906308, L_x: 1.0308029386365294,  L_y: 0.4435891757918725,  L_z: 0.007643185267552722
step: 1750, loss: 0.5585636756338709, L_x: 1.0296490241047005,  L_y: 0.44074622164810573,  L_z: 0.007426275787647554
step: 2000, loss: 0.558

2024-01-02 14:18:26 experiment_interface.py INFO    : The dataset is preprocessed
INFO:root:The dataset is preprocessed
2024-01-02 14:18:26 experiment_interface.py INFO    : Models config is loaded from the input file
INFO:root:Models config is loaded from the input file


...

[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGB

[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGB

[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGB

Classifiers testing by bootstrap:   0%|          | 0/200 [00:00<?, ?it/s]

Classifiers testing by bootstrap:   0%|          | 0/200 [00:00<?, ?it/s]